# **Segmenting and Clustering Neighborhoods in Toronto**


Import the required modules and functions


In [1]:
#!pip install bs4
#!pip install requests

In [2]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

## Scrape data from Toronto Wiki page


In [3]:
#The below url contains the neighborhoods in Toronto.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Scrape data from wiki site

In [4]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [5]:
soup = BeautifulSoup(data,"html5lib") #create a beautiful soup object "soup"

In [6]:
#find html table in the web page
table = soup.find('table') # html table is represented by the tag <table>

In [7]:
# #Get all Toronto postal code rows from the table and parse into "torr_data" list
tor_data = []  
for row in table.find_all('td'): # each new record has the tag <td>
    cell = {}
    if row.span.text=='Not assigned': # skip 'Not assigned' records
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        tor_data.append(cell)    
print(tor_data)    

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

## 1. Move the list into a DataFrame using Pandas and get shape


In [8]:
import pandas as pd

In [9]:
#Place into Pandas Dataframe
df =pd.DataFrame(tor_data)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
# clean up some borough names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [11]:
df.shape

(103, 3)

## 2. GeoCode CSV and merge with Tor data frame using Pandas


In [12]:
# Read the csv into Pandas
GeoData=pd.read_csv("Geospatial_Coordinates.csv")

In [13]:
# Set GeoData index on 'Postal Code' to merge with Torr data
GeoData.set_index('Postal Code', inplace=True)
GeoData.head()
# Set df index on 'PostalCode' to merge with GeoData
df.set_index('PostalCode', inplace=True)

In [14]:
# Merge to two dataframes to a new Dataframe containing all the data and indexed on Postal Code
TorDf=df.merge(GeoData, left_index=True, right_index=True)

In [15]:
TorDf.reset_index(inplace=True)

In [16]:
TorDf.head()


,index,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
